In [93]:
from __future__ import division
import numpy as np
from tqdm import tqdm
import matplotlib
matplotlib.use('Agg')
# You need to convert them into .py files 
import os
import sys
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# %matplotlib inline
# %precision 4
plt.style.use('ggplot')

np.random.seed(1234)
import scipy.stats as stats
import time

def calcM(Z,Kplus,sigmaX,sigmaA):
    """Save the matrix M so we won't need to calculate it again and again"""
    try:
        Kplus = min(Kplus, Z.shape[1])
        a = np.linalg.inv(np.dot(Z[:,0:Kplus].T,Z[:,0:Kplus])+((sigmaX/sigmaA)**2)*np.identity(Kplus))
    except Exception as e:
        print(Z.shape, Kplus, sigmaX, sigmaA)
    return a

def calcInverse_orig(Z, M, i, k, val):
    """Effective inverse calculation from Griffiths and Ghahramani (2005; Equations 51 to 54)
    M_(-i) = inv(inv(M) - zi.T * zi)"""
    M_i = M - np.dot(np.dot(M,Z[i,:].T),np.dot(Z[i,:],M))/(np.dot(np.dot(Z[i,:],M),Z[i,:].T)-1)
    Z[i,k] = val
    M = M_i - np.dot(np.dot(M_i,Z[i,:].T),np.dot(Z[i,:],M_i))/(np.dot(np.dot(Z[i,:],M_i),Z[i,:].T)+1)
    Inv = M
    return Inv

def calcInverse(Z, M, i, k, val):
    """New version to check: M_(-i) = inv(inv(M) - zi.T * zi) and M = inv(inv(M_(-i)) + zi.T * zi)"""
    M_i = np.linalg.inv(np.linalg.inv(M) - np.dot(Z[i,:].T,Z[i,:]))
    Z[i,k] = val
    M = np.linalg.inv(np.linalg.inv(M_i) + np.dot(Z[i,:].T,Z[i,:]))
    return M

def log_likelihood(X,Z,M,sigmaA,sigmaX,Kplus,N,D):  
    """Calculate the log-likelihood: P(X|Z,M,sigmaA,sigmaX,Kplus,N,D)"""  
    determinant = np.linalg.det(np.dot(Z.T,Z)+((sigmaX/sigmaA)**2)*np.identity(Z.shape[1]))
    constant = N*D*0.5*np.log(2*np.pi) + (N-Kplus)*D*np.log(sigmaX) + Kplus*D*np.log(sigmaA) + D*0.5*np.log(determinant)
    
    middle = np.identity(N) - np.dot(np.dot(Z, M),Z.T)
    trace = np.trace(np.dot(np.dot(X.T,middle),X))
    kernel = -0.5*np.reciprocal(sigmaX**2)*trace
    
    log_lik = -constant + kernel
    return log_lik

In [94]:
def IBP(N, alpha):
    """Indian Buffet Process (IBP) steps:
    Input: N is the number of customers (objects, images); alpha is the only parameter;
    Return: result is the binary matrix (prior); Kplus is the number of dishes (features)"""
    result = np.zeros((N,1000))
    
    # Step 1: First customer takes a Poisson(alpha) of dishes
    t = stats.poisson.rvs(alpha) # (set the random seed when calling the function)
    if t > 0:
        result[0,0:t] = 1
    
    # Kplus = the number of features for which m_k > 0 (m_k: the number of previous customers who sampled that dish)
    Kplus = t
    for i in range(1,N):
        for k in range(Kplus):
            # Step 2: The ith customer takes dish k with probability m_k/i
            p = np.sum(result[0:(i+1),k])/(i+1) # this is a probability, so should be between 0 and 1
            assert p <= 1 
            assert p >= 0
            if stats.uniform.rvs(0) < p:
                result[i,k] = 1
            else:
                result[i,k] = 0
                
        # Step 3: The ith customer tries a Poisson(alpha/i) number of new dishes
        t = stats.poisson.rvs(alpha/(i+1))
        if t > 0:
            result[i,Kplus:(Kplus+t)] = 1
        Kplus += t
    result = result[:,0:Kplus]
    
    return result, Kplus

In [104]:
def updateZ(i, k, Z, Kplus,N,D,sigmaA, sigmaX,alpha,K_inf):   
    # This is possible because Kplus may decrease in this loop (e.g. dropping redundant zeros)
    if (k+1) > Kplus or k >= Z.shape[1]:
        return False
    if Z[i,k] > 0:
        # Take care of singular features
        # Get rid of the features not sampled (remove the zeros)
        if np.sum(Z[:,k]) - Z[i,k] <= 0: # whether the dish is sampled by other customers or not
            Z[:,k:(Kplus-1)] = Z[:,(k+1):Kplus]
            Kplus -= 1    
            return True            
    
    # Effective inverse calculation from Griffiths and Ghahramani (2005; Equations 51 to 54)
    # M_(-i) = inv(inv(M) - zi.T * zi)
    # Then calculate the posterior distribution: prior * likelihood 
    P = np.zeros(2)
    Z[i,k] = 1
    M1 = calcM(Z,Kplus,sigmaX,sigmaA) 
    P[1] = log_likelihood(X,Z[:,0:Kplus],M1,sigmaA,sigmaX,Kplus,N,D) + np.log(sum(Z[:,k])-Z[i,k]) - np.log(N)
    Z[i,k] = 0
    M0 = calcM(Z,Kplus,sigmaX,sigmaA) 
    P[0] = log_likelihood(X,Z[:,0:Kplus],M0,sigmaA,sigmaX,Kplus,N,D) + np.log(N-sum(Z[:,k])) - np.log(N)
    P = np.exp(P - max(P))
    # Sample from the posterior distribution
    rand = stats.uniform.rvs(loc=0,scale=1,size=1)           
    if rand < P[0]/(P[0]+P[1]):
        Z[i,k] = 0
        #M = M0
    else:
        Z[i,k] = 1
    return True

In [112]:
def updateK(i, Z,Kplus,N,D,sigmaA, sigmaX,alpha,K_inf):
    trun = np.zeros(4)
    alphaN = alpha / N     
    
    for ki in range(4):
        if ki > 0:
            new_stack = np.zeros((N,ki))
            new_stack[i,:] = 1
            Z = np.hstack((Z[:,0:Kplus],new_stack))
        M = np.linalg.inv(np.dot(Z[:,0:(Kplus+ki)].T,Z[:,0:(Kplus+ki)])
        M += ((sigmaX/sigmaA)**2)*np.identity(M.shape[1]))
        # Prior: x ~ Pois(lambda): f(x) = ((lambda**x)/x!)*exp(-lambda), where x = ki, lambda = alphaN
        
        trun[ki] = (ki)*np.log(alphaN) - alphaN - np.log(np.math.factorial(ki)) 
        # posterior is proportional to prior x likelihood
        trun[ki] += log_likelihood(X,Z[:,0:(Kplus+ki)],M,sigmaA,sigmaX,Kplus+ki,N,D)
        
    # Z[i,Kplus:(Kplus+4)] = 0
    Z[i,Kplus:(Kplus+3)] = 0
    trun = np.exp(trun-max(trun))
    trun = trun/np.sum(trun)
    
    p = stats.uniform.rvs(loc=0,scale=1,size=1)  
    t = 0
    # for ki in range(5):
    for ki in range(4):
        t += trun[ki]
        if p < t:
            new_dishes = ki
            break
    Z[i,Kplus:(Kplus+new_dishes)] = 1
    Kplus += new_dishes
    return Kplus

SyntaxError: invalid syntax (<ipython-input-112-ddab4bfecec1>, line 11)

In [113]:
def update_sigma_X(Z,Kplus,N,D,sigmaA, sigmaX,alpha,K_inf,logLik):
    epsilonX = stats.uniform.rvs(loc=0,scale=1,size=1) 
    if epsilonX < 0.5:
        # sigmaX_star = sigmaX - epsilonX/40
        # sigmaX_star = sigmaX - epsilonX/20
        sigmaX_star = sigmaX - stats.uniform.rvs(loc=0,scale=1,size=1)/20
    else:
        # sigmaX_star = sigmaX + epsilonX/20   
        sigmaX_star = sigmaX + stats.uniform.rvs(loc=0,scale=1,size=1)/20 
    # M_Xstar = calcM(Z, Kplus+new_dishes, sigmaX_star, sigmaA)
    M_Xstar = calcM(Z, Kplus, sigmaX_star, sigmaA)
    # logLikX_star = log_likelihood(X, Z[:,0:(Kplus+new_dishes)], M_Xstar, sigmaA, sigmaX_star, Kplus+new_dishes, N, D)
    logLikX_star = log_likelihood(X, Z[:,0:Kplus], M_Xstar, sigmaA, sigmaX_star, Kplus, N, D)
    acc_X = np.exp(min(0, logLikX_star-logLik))
    randX = stats.uniform.rvs(loc=0,scale=1,size=1)
    if randX < acc_X:
        sigmaX = sigmaX_star
    return sigmaX

In [114]:
def update_sigma_A(Z,Kplus,N,D,sigmaA, sigmaX,alpha,K_inf,logLik):
    epsilonA = stats.uniform.rvs(loc=0,scale=1,size=1)
    if epsilonA < 0.5:
        # sigmaA_star = sigmaA - epsilonA/20
        sigmaA_star = sigmaA - stats.uniform.rvs(loc=0,scale=1,size=1)/20
    else:
        # sigmaA_star = sigmaA + epsilonA/40 
        sigmaA_star = sigmaA + stats.uniform.rvs(loc=0,scale=1,size=1)/20
        # sigmaA_star = sigmaA + epsilonA/20   
    # M_Astar = calcM(Z, Kplus+new_dishes, sigmaX, sigmaA_star)
    M_Astar = calcM(Z, Kplus, sigmaX, sigmaA_star)
    # logLikA_star = log_likelihood(X, Z[:,0:(Kplus+new_dishes)], M_Astar, sigmaA_star, sigmaX, Kplus+new_dishes, N, D)
    logLikA_star = log_likelihood(X, Z[:,0:Kplus], M_Astar, sigmaA_star, sigmaX, Kplus, N, D)
    acc_A = np.exp(min(0, logLikA_star-logLik))
    
    randA = stats.uniform.rvs(loc=0,scale=1,size=1)
    if randA < acc_A:
        sigmaA = sigmaA_star
    return sigmaA

In [115]:
def Gibbs(X, sigmaA, sigmaX, alpha, max_iter, K_inf):
    time_total = 0
    time_Z = [0,0]
    time_K = [0,0]
    time_alpha = [0,0]
    time_sigma_X = [0,0]
    time_sigma_A = [0,0]
    __t = time.time()
    N, D = X.shape
    Z, Kplus = IBP(N, alpha)
    from fractions import Fraction
    sum_Harmonics = 0
    Harmonics = 0
    for i in range(N):
        sum_Harmonics += (N-i)*Fraction(1,i+1)
        Harmonics += Fraction(1,i+1)
    with tqdm(total = max_iter) as pbar:
        for mc in range(max_iter): # just test for 10 iterations
            # Step 1: Generate Z|alpha (Gibbs)
            for i in range(N):
                for k in range(Kplus):
                    _t = time.time()      
                    a = updateZ(i,k,Z,Kplus,N,D,sigmaA, sigmaX,alpha,K_inf)
                    time_Z[0] += time.time() - _t
                    time_Z[1] += 1
                    if a == False:
                        break
                # Sample the number of new dishes Pois(alpha/i) for the current customer/object
                _t = time.time()
                Kplus = min(updateK(i, Z,Kplus,N,D,sigmaA, sigmaX,alpha,K_inf), Z.shape[0])
                time_K[0] += time.time() - _t
                time_K[1] += 1
                
            M = calcM(Z, Kplus, sigmaX, sigmaA)
            logLik = log_likelihood(X, Z[:,0:Kplus], M, sigmaA, sigmaX, Kplus, N, D)
            
            # Step 2: Sample sigmaX_star (Metropolis) 
            _t = time.time()
            sigmaX = update_sigma_X(Z,Kplus,N,D,sigmaA, sigmaX,alpha,K_inf,logLik)
            time_sigma_X[0] += time.time() - _t
            time_sigma_X[1] += 1
            
            # Step 3: Sample sigmaA_star (Metropolis)
            _t = time.time()
            sigmaA = update_sigma_A(Z,Kplus,N,D,sigmaA, sigmaX,alpha,K_inf,logLik)
            time_sigma_A[0] += time.time() - _t
            time_sigma_A[1] += 1
            
            # Step 4: Sample alpha|Z ~ Ga(a=1+Kplus,scale=1+Harmonics)
            _t = time.time()
            alpha = stats.gamma.rvs(a = 1+Kplus, loc = 0, scale = np.reciprocal(1+Harmonics),size=1)[0]
            time_alpha[0] += time.time() - _t
            time_alpha[1] += 1
            pbar.set_description("Current K = %s" % Kplus)
            pbar.update(1)
    time_total += time.time() - __t
    return time_Z, time_K, time_alpha, time_sigma_X, time_sigma_A, time_total

In [121]:
def profile(args):
    time_Z, time_K, time_alpha, time_sigma_X, time_sigma_A, time_total = args
    print(f"Total time used: {time_total}s")
    print(f"Update Z\ttot_time {time_Z[0]}s\t#exec {time_Z[1]}\tavg_time {time_Z[0]/time_Z[1]}s")
    print(f"Update K\ttot_time {time_K[0]}s\t#exec {time_K[1]}\tavg_time {time_K[0]/time_K[1]}s")
    print(f"Update alpha\ttot_time {time_alpha[0]}s\t#exec {time_alpha[1]}\tavg_time {time_alpha[0]/time_alpha[1]}s")
    print(f"Update sigma_X\ttot_time {time_sigma_X[0]}s\t#exec {time_sigma_X[1]}\tavg_time {time_sigma_X[0]/time_sigma_X[1]}s")
    print(f"Update sigma_A\ttot_time {time_sigma_A[0]}s\t#exec {time_sigma_A[1]}\tavg_time {time_sigma_A[0]/time_sigma_A[1]}s")
    print(f"Other\t\ttot_time {time_total - time_Z[0] - time_K[0] - time_alpha[0] - time_sigma_X[0] - time_sigma_A[0]}s")
    return list(args)


In [133]:
V1 = np.array([
    1, 0, 0, 0, 0, 0,
    1, 0, 0, 0, 0, 0,
    1, 0, 0, 0, 0, 0,
    1, 0, 0, 0, 0, 0,
    0, 0, 0, 0, 0, 0,
    0, 0, 0, 0, 0, 0
])
V2 = np.array([
    0, 0, 0, 0, 0, 0,
    0, 0, 0, 0, 0, 0,
    0, 0, 0, 0, 0, 0,
    0, 0, 0, 0, 0, 0,
    1, 0, 0, 0, 0, 0,
    1, 1, 1, 0, 0, 0
])
V3 = np.array([
    0, 0, 0, 0, 0, 0,
    0, 0, 0, 0, 0, 0,
    0, 0, 0, 0, 0, 0,
    0, 0, 0, 0, 0, 0,
    0, 0, 0, 0, 1, 1,
    0, 0, 0, 0, 1, 1
])
V4 = np.array([
    0, 0, 0, 0, 1, 1,
    0, 0, 0, 0, 0, 1,
    0, 0, 0, 0, 0, 1,
    0, 0, 0, 0, 0, 0,
    0, 0, 0, 0, 0, 0,
    0, 0, 0, 0, 0, 0
])
V5 = np.array([
    0, 0, 0, 0, 0, 0,
    0, 0, 0, 0, 0, 0,
    0, 1, 1, 1, 0, 0,
    0, 0, 1, 0, 0, 0,
    0, 0, 0, 0, 0, 0,
    0, 0, 0, 0, 0, 0
])
V = [V1, V2, V3, V4, V5]

N = 100
D1 = np.outer(np.random.binomial(1, 0.5, N), V1)
D2 = np.outer(np.random.binomial(1, 0.5, N), V2)
D3 = np.outer(np.random.binomial(1, 0.5, N), V3)
D4 = np.outer(np.random.binomial(1, 0.5, N), V4)
D5 = np.outer(np.random.binomial(1, 0.5, N), V5)
X = D1 + D2 + D3 + D4 + D5
X = X[np.sum(X, axis = 1) > 0] * 1.0
X += np.random.normal(0, 0.1, X.shape)
print(X.shape)

(97, 36)


In [136]:
# Initialization
sigmaA = 1
sigmaX = 1
alpha = 1

g = Gibbs(X, sigmaA, sigmaX, alpha, max_iter = 1000, K_inf = 1000)

Current K = 10: 100%|██████████| 1000/1000 [08:56<00:00,  1.78it/s]


In [137]:
profile(g)

Total time used: 536.5724658966064s
Update Z	tot_time 450.75351667404175s	#exec 970000	avg_time 0.000464694347086641s
Update K	tot_time 82.61395215988159s	#exec 97000	avg_time 0.0008516902284523875s
Update alpha	tot_time 0.13987135887145996s	#exec 1000	avg_time 0.00013987135887145996s
Update sigma_X	tot_time 0.3475663661956787s	#exec 1000	avg_time 0.0003475663661956787s
Update sigma_A	tot_time 0.33116912841796875s	#exec 1000	avg_time 0.0003311691284179687s
Other		tot_time 2.386390209197998s


[[450.75351667404175, 970000],
 [82.61395215988159, 97000],
 [0.13987135887145996, 1000],
 [0.3475663661956787, 1000],
 [0.33116912841796875, 1000],
 536.5724658966064]

In [123]:
def to_black(path):
    fig = plt.imread(path)[:, :, :3]
    print(fig.shape)
    fig = np.mean(fig, axis = -1)
    #fig[fig < 0.5] = 0
    #fig[fig >= 0.5] = 1
    return 1 - fig

def flatten(fig):
    return fig.reshape(-1)

def get_diamond():
    num = ['A', '2', '3', '4', '5', '6', '7', '8', '9', '10']
    paths = ['poker/' + n + 'D.png' for n in num]
    imgs = [to_black(path) for path in paths]
    new_img = np.zeros((1056,691))
    new_img[2:-2, 4:-4] = imgs[6]
    imgs[6] = new_img
    return np.stack([flatten(img[130:-130:10,130:-130:10]) for img in imgs])

Xd = get_diamond()
X = np.tile(Xd, reps = (10, 1))
X += np.random.normal(0, 0.15, X.shape)
X_avg = np.mean(X, axis=0)
X -= X_avg

sigmaA = 0.5
sigmaX = 0.1
alpha = 1

g = Gibbs(X, sigmaA, sigmaX, alpha, max_iter = 300, K_inf = 1000)

(1056, 691, 3)
(1056, 691, 3)
(1056, 691, 3)
(1056, 691, 3)
(1056, 691, 3)
(1056, 691, 3)
(1052, 683, 3)
(1056, 691, 3)
(1056, 691, 3)
  0%|          | 0/300 [00:00<?, ?it/s](1056, 691, 3)



ValueError: operands could not be broadcast together with shapes (3,3) (4,4) 